# Visualize Satellite Data

Need Thailand province, district, tambon shape files. Get one from 
https://data.humdata.org/dataset/thailand-administrative-boundaries

In [ ]:
import pandas as pd
import geopandas as gpd
import branca
import folium
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [ ]:
directory = 'DOAE spatial data/ดาวเทียม/'
rice_file = directory + '20200229_16_Rice/rice_20200229.shp'

In [ ]:
rice = gpd.read_file(rice_file, encoding='utf-8')

In [ ]:
rice.head()

In [ ]:
rice.columns

In [ ]:
rice[['p_name', 'a_name', 'yield', 'yield_m','start_date', 'start_name', 'harv_date',
       'harv_name', 'crop_type']].head()

In [ ]:
rice_plot_data = rice[['p_name', 'yield', 'Shape_Area']].groupby('p_name').sum().reset_index()

In [ ]:
rice_plot_data

## Thailand Map

In [ ]:
province_map_file = './tha_adm_rtsd_itos_20190221_shp_part_1/tha_admbnda_adm1_rtsd_20190221.shp'

In [ ]:
province_map = gpd.read_file(province_map_file, encoding='utf-8')

In [ ]:
province_map.columns

In [ ]:
geo_data = province_map[['ADM1_TH', 'ADM0_TH','geometry']]

In [ ]:
geo_data.head()

In [ ]:
rice_data = geo_data.merge(rice_plot_data, left_on='ADM1_TH', right_on='p_name')

In [ ]:
rice_data

In [ ]:
rice_data['yield'] = rice_data['yield'].round()

In [ ]:
colormap = branca.colormap.LinearColormap(vmin=rice_data['yield'].min(),
                                          vmax=rice_data['yield'].max(),
                                          colors=['red','orange','lightblue','green','darkgreen'],
                                          caption="Rice Yield")

In [ ]:
m = folium.Map(location=[14.069446, 100.603271], zoom_start=4)

popup=GeoJsonPopup(fields=['p_name','yield','Shape_Area'], aliases=['จังหวัด','ผลผลิต','ขนาด'], localize=True, labels=True, style="background-color: yellow;")

tooltip=GeoJsonTooltip(
    fields=["p_name", "yield"],
    aliases=["Province:", "Yield:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)


g = folium.GeoJson(
    rice_data,
    style_function=lambda x: {
        "fillColor": colormap(x["properties"]["yield"])
            if x["properties"]["yield"] is not None
            else "transparent",
        "color": "black",
        "weight": 0.5,
        "fillOpacity": 0.4,
    },
    tooltip=tooltip,
    popup=popup
).add_to(m)

colormap.add_to(m)

m

In [ ]:
m.save('./output/sat_viz2.html')